# Databased used
- CREMA-D: 7,442 clips
- RAVDESS: 2,880 clips
- TESS: 2,000 clips
- SAVEE: 480 clips

This provides a total of **12,802** inputs overall, with 7 emotions.

In [ ]:
# Import libraries
# KERNEL SETUP IN VS CODE:
# conda create -n myenv python=3.12.2
# conda activate myenv

%pip install resampy tf_keras tensorflow librosa pandas matplotlib kagglehub

import kagglehub
import librosa
from librosa import feature
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os

In [2]:
# Datasets via kagglehub

cremad = kagglehub.dataset_download("ejlok1/cremad")
print("CREMA-D to dataset files:", cremad)

ravdess = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio")
print("RAVDESS to dataset files:", ravdess)

tess = kagglehub.dataset_download("ejlok1/toronto-emotional-speech-set-tess")
print("RAVDESS to dataset files:", tess)

savee = kagglehub.dataset_download("ejlok1/surrey-audiovisual-expressed-emotion-savee")
print("RAVDESS to dataset files:", savee)

CREMA-D to dataset files: C:\Users\eskillman\.cache\kagglehub\datasets\ejlok1\cremad\versions\1


100%|██████████| 429M/429M [00:09<00:00, 46.5MB/s] 

Extracting files...


RAVDESS to dataset files: C:\Users\eskillman\.cache\kagglehub\datasets\uwrfkaggler\ravdess-emotional-speech-audio\versions\1


100%|██████████| 428M/428M [00:09<00:00, 49.5MB/s] 

Extracting files...


RAVDESS to dataset files: C:\Users\eskillman\.cache\kagglehub\datasets\ejlok1\toronto-emotional-speech-set-tess\versions\1


100%|██████████| 107M/107M [00:08<00:00, 13.7MB/s] 

Extracting files...


RAVDESS to dataset files: C:\Users\eskillman\.cache\kagglehub\datasets\ejlok1\surrey-audiovisual-expressed-emotion-savee\versions\1


# CREMA-D

In [ ]:
# Load CREMA-D Dataset
paths = []
emotions = []

for dirname, _, filenames in os.walk(cremad): # (dirname, subdirs, filenames)
    for filename in filenames:
        if filename.endswith('.wav'):
            paths.append(os.path.join(dirname, filename))
            emotion = filename.split('_')[2]  # Get the emotion code (e.g., 'ANG')
            emotions.append(emotion)

print(paths[:5])

print(emotions[:5])

# Create DataFrame
creamd_df = pd.DataFrame()
creamd_df['speech'] = paths
creamd_df['emotions'] = emotions

# Map emotion codes to full emotions
emotion_map = {
    'ANG': 'anger',
    'DIS': 'disgust',
    'FEA': 'fear',
    'HAP': 'happy',
    'NEU': 'neutral',
    'SAD': 'sad'
}

creamd_df['emotions'] = creamd_df['emotions'].map(emotion_map)

print(creamd_df.head())

print(creamd_df['emotions'].value_counts())

['C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\ejlok1\\cremad\\versions\\1\\AudioWAV\\1001_DFA_ANG_XX.wav', 'C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\ejlok1\\cremad\\versions\\1\\AudioWAV\\1001_DFA_DIS_XX.wav', 'C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\ejlok1\\cremad\\versions\\1\\AudioWAV\\1001_DFA_FEA_XX.wav', 'C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\ejlok1\\cremad\\versions\\1\\AudioWAV\\1001_DFA_HAP_XX.wav', 'C:\\Users\\eskillman\\.cache\\kagglehub\\datasets\\ejlok1\\cremad\\versions\\1\\AudioWAV\\1001_DFA_NEU_XX.wav']
['ANG', 'DIS', 'FEA', 'HAP', 'NEU']


KeyError: 'emotions'